# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 1 (Holder) 🚗

In [ ]:
%%javascript
document.title = '🚗 Manufacturer1'

## PART 3: Connect with City to Analyze Data

**What:** Share encrypted data with City agent in a trust- and privacy-preserving manner

**Why:** Share data with City agent (e.g., to obtain funds)

**How:** <br>
1. [Initiate Manufacturer's AgentCommunicationManager (ACM)](#1)
2. [Connect anonymously with the City agent via a multi-use SSI invitation](#2)
3. [Prove Manufacturer1 Agent is a certified manufacturer via VCs](#3)
4. [Establish anonymous Duet Connection to share encrypted data](#4)

**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_manufacturer1.ipynb`
* Optional – Manufacturer2 🚛: `03_connect_with_city.ipynb`
* Optional – Manufacturer3 🛵: `03_connect_with_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [ ]:
import os

import numpy as np
import pandas as pd
import syft as sy
import torch
from aries_cloudcontroller import AriesAgentController

from libs.agent_connection_manager import CredentialHolder

#### 0.2 – Variables

In [ ]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer1 Agent
#### 1.1 – Init ACA-PY agent controller

In [ ]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [ ]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

#### 1.3 – Init ACM Credential Holder

In [ ]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer1_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer1_agent.get_credentials()

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the manufacturer1 requests a connection with the Authority to be certified as an official city agency. Thus, the manufacturer1 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [ ]:
# Variables
alias = "undisclosedM1"
auto_accept = True

# Receive connection invitation
connection_id = manufacturer1_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

<div style="font-size: 25px"><center><b>Break Point 2 / 3 / 4</b></center></div>
<div style="font-size: 50px"><center>🚗 ➡️ 🚛 / 🛵 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.2</b></center>

---

<a id=3></a>
### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer1 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [ ]:
presentation, presentation_exchange_id = manufacturer1_agent.prepare_presentation(connection_id)

#### 3.2 – Send Presentation

Overall, the proof-presentation procedure has six steps. **R** represents the Relying Party (here the City agent), and **H** the Holder (i.e., the manufacturers).

| Step | Status | Agent | Description | Function |
| --- | --- | --- | --- | --- |
| 1 | `request_sent` |  R | R requests a proof defined in `manufacturer_proof_request` | `send_proof_request()` |
| 2 | `request_received` | H | H receives a proof request from R | - |
| 3 | `presentation-sent` | H | H sends proof presentation to R | `send_proof_presentation()` |
| 4 | `presentation-received` | R  | R receives presentation from H | - |
| 5 | `verified` | R | R verifies presentation received from H | `verify_proof_presentation()` |
| 6 | `presentation_acked` | H | H knows, that R verified the presentation | - |


Now, send the presentation to the recipient of `presentation_exchange_id`. 

In [ ]:
manufacturer1_agent.send_proof_presentation(presentation_exchange_id, presentation)

---
<a id=4></a>
### 4 – Do Data Science
Assuming that the City agent will acknowledge the proofs and deem them to be correct, proceed by inviting the City agent to a Duet Connection.


#### 4.1 – Establish a Duet Connection with City Agent: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [ ]:
# Set up connection_id to use for duet connection
manufacturer1_agent._update_connection(connection_id=connection_id, is_duet_connection=True, reset_duet=True)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer1_agent)

<div style="font-size: 25px"><center><b>Break Point 6 / 7 / 8</b></center></div>
<div style="font-size: 50px"><center>🚗 ➡️ 🚛 / 🛵 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>

#### 4.2 - Load data to duet store

In [ ]:
# Verify data store of duet
duet.store.pandas  # There should only be an MPC session statement by the City agent

Process data before loading it to the duet store. We take a synthetically created dataset of CO2 emission per trip across the City Agent's City (in this case Berlin, Germany).

In [ ]:
# Get zipcode data (zipcode data from https://daten.odis-berlin.de/de/dataset/plz/)
df_zipcode = pd.read_csv("data/geo/berlin_zipcodes.csv").rename(columns={"pls":"zipcode"})
df_zipcode.head()

In [ ]:
# Get trip data
df_co2 = pd.read_csv("data/trips/data.csv", index_col=0)
df_co2["hour"] = df_co2.timestamp.apply(lambda x: int(x[11:13]))
df_co2.head()

The trip data is then grouped by zipcode to sum the CO2 emission per hour per zipcode.


In [ ]:
# Get hourly co2
df_hourly_co2 = df_co2[["zipcode", "hour","co2_grams"]].groupby(["zipcode", "hour"]).sum().reset_index()
df_hourly_co2 = df_hourly_co2.pivot(index=["zipcode"], columns=["hour"])["co2_grams"].replace(np.nan, 0)

# Get matrix that of shape (4085,25)
df_hourly_zipcode = df_zipcode.set_index("zipcode").reindex(columns=list(range(0,24))).replace(np.nan,0)#.reset_index()

# Merge dataframes together
df = df_hourly_zipcode.add(df_hourly_co2, fill_value=0)
print(df.shape)
df.head()

Then, convert the dataset to a tensor, and upload the tensor with shape (194 x 24) to the duet data store

In [ ]:
# Configure tensor
hourly_co2_torch = torch.tensor(df.values)
hourly_co2_torch = hourly_co2_torch.tag("hourly-co2-per-zip_2021-08-19")
hourly_co2_torch = hourly_co2_torch.describe("Total CO2 per Zipcode per Hour on August 19, 2021. Shape: zipcode (10115-14199) x hour (0-23) = 4085 x 24")

# Load tensor to datastore
hourly_co2_torch_pointer = hourly_co2_torch.send(duet, pointable=True)

# Verify datastore
duet.store.pandas

#### 4.3 – Authorize City agent to `.reconstruct()` the data
Authorize the city agent to reconstruct the data once it is shared and joined with other manufacutrers' data.

In [ ]:
duet.requests.add_handler(
    name="reconstruct",
    action="accept"
)

---

### 5 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.
(Note: the terminating the controller will not terminate the Duet session).

In [ ]:
await agent_controller.terminate()

---

<div style="font-size: 25px"><center><b>Break Point 10 / 11 / 12</b></center></div>
<div style="font-size: 50px"><center>🚗 ➡️ 🚛 / 🛵 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 4.2 and 4.3 <br> If you have uploaded all data to the manufacturers' datastored, proceed to the City agent Step 4.2</b></center>

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥